# Lectura 44: Agregaciones y funciones de ventana

## Agregaciones

### Condicionales

Digamos que queremos saber cuáles fueron los tres aeropuertos de origen con más vuelos para la aerolínea American Airlines (AA).

In [ ]:
import polars as pl

vuelos_lf = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
)

In [ ]:
from polars import col

(
    vuelos_lf
    .group_by('ORIGIN_AIRPORT')
    .agg(
        (col('AIRLINE') == 'AA').sum().alias('AA')
    )
    .sort('AA', descending=True)
    .limit(3)
    .collect()
)

## Funciones de ventana

Las funciones de ventana en Polars nos permiten realizar agregaciones en grupos dentro de un `select`. 

A continuación, mostraremos cómo usar funciones de ventana para agrupar diferentes columnas y realizar una agregación en ellas. Hacerlo nos permite utilizar múltiples operaciones de `group by` en paralelo, utilizando una única consulta. Los resultados de la agregación se proyectan a las filas originales. Por lo tanto, una función de ventana casi siempre conducirá a un DataFrame del mismo tamaño que el original.

Para poder acceder a las funciones de ventana podemos llamar al método `over('col1')` o si deseamos más de una columna lo podemos hacer llamando a `over(['col1', 'col2', ..., colN])` .

Al usar  funciones de ventana podemos realizar agregaciones sobre diferentes grupos dentro de un solo `select`. Esto no costará nada porque los grupos calculados se almacenan en caché y se comparten entre diferentes expresiones de ventana.

En el siguiente ejemplo vamos a calcular primeramente el promedio del tiempo en el aire por aerolínea y luego calcularemos el promedio del retraso en la salida por aeropuerto origen y aeropuerto destino.

In [ ]:
vuelos_lf.select(
    col('AIRLINE'),
    col('AIR_TIME'),
    col('AIR_TIME').mean().over('AIRLINE').alias('AVG_AIR_TIME'),
    col('ORIGIN_AIRPORT'),
    col('DESTINATION_AIRPORT'),
    col('DEPARTURE_DELAY').mean().over(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']).alias('AVG_DEPARTURE_DELAY')   
).collect()

### Operaciones por grupo

Las funciones de ventana pueden hacer más que solo agregaciones. También pueden verse como una operación dentro de un grupo. Si, por ejemplo, deseamos ordenar los valores dentro de un grupo podemos realizarlo.

Para mostrar esto vamos a tomar del dataset de vuelos un subconjunto de columnas. Particionaremos por el día de la semana y ordenaremos por los retrasos en las llegadas dentro de esos grupos.

In [ ]:
vuelos_reducido_lf = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .select(
        col('AIRLINE'),
        col('DAY_OF_WEEK'),
        col('ARRIVAL_DELAY')
    )
)

In [ ]:
vuelos_reducido_lf.collect()

In [ ]:
vuelos_reducido_lf.with_columns(
    col(['AIRLINE', 'ARRIVAL_DELAY']).sort_by('ARRIVAL_DELAY').over('DAY_OF_WEEK')
).collect()